In [2]:
import torch

# Supplement : getIdentity

In [18]:
def get_identity(B, idB, *args):
    """
    Get the identity tensor in the space of the idB-th leg of B.
    If C and idC are provided, get the identity tensor in the direct 
    product space of the Hilbert space of the idB-th leg of B and 
    the space of the idC-th leg of C.
    
    Args:
    B: Tensor
    idB: integer, index for B
    C: Tensor, optional
    idC: integer, optional, index for C
    p: list of integers, optional, permutation of the contracted tensor 

    Returns:
    A: Tensor, identity tensor
    """
    # parsing input
    C = None
    idC = None
    p = None

    if len(args) > 1:  # combine the spaces of two tensors
        C = args[0]
        idC = args[1]
        if len(args) > 2:
            p = args[2]
    elif len(args) == 1:  # consider only one space
        p = args[0]

    DB = B.shape[idB]
    if C is not None:
        DC = C.shape[idC]
        A = torch.eye(DB*DC).reshape([DB, DC, DB*DC])
    else:
        A = torch.eye(DB)

    if p is not None:
        if len(p) < len(A.shape):
            raise ValueError("The number of elements of permutation option 'p' is smaller than the rank of 'A'.")
        A = A.permute(p)

    return A

# Supplement : getLocalSpace

In [19]:
def get_local_space(*args):
    """
    Generate local operators as tensors based on the input parameters.

    Args:
    args: The type of tensor to generate. Valid inputs include: 'Spin', 'Fermion', 'FermionS'

    Returns:
    A list of tensors based on the input parameters.
    """
    if len(args) == 0 or args[0] not in ['Spin', 'Fermion', 'FermionS']:
        raise ValueError("Input #1 should be either 'Spin', 'Fermion', or 'FermionS'.")

    is_fermion = False
    is_spin = False
    I = None
    s = 0

    if args[0] == 'Spin':
        if len(args) < 2:
            raise ValueError("For 'Spin', input #2 is required.")
        s = args[1]
        if (abs(2 * s - round(2 * s)) > 1e-9) or (s <= 0):
            raise ValueError("Input #2 for 'Spin' should be positive (half-)integer.")
        s = round(2 * s) / 2
        is_spin = True  # create S tensor
        I = torch.eye(int(2 * s + 1))

    elif args[0] == 'Fermion':
        is_fermion = True  # create F and Z tensors
        I = torch.eye(2)

    elif args[0] == 'FermionS':
        is_fermion = True
        is_spin = True
        I = torch.eye(4)

    if is_fermion:
        if is_spin:  # spinful fermion
            # basis: empty, up, down, two (= c_down^+ c_up^+ |vac>)
            F = torch.zeros(4, 4, 2)
            # spin-up annihilation
            F[0, 1, 0] = 1
            F[2, 3, 0] = -1  # -1 sign due to anticommutation
            # spin-down annihilation
            F[0, 2, 1] = 1
            F[1, 3, 1] = 1

            Z = torch.diag(torch.tensor([1, -1, -1, 1]))

            S = torch.zeros(4, 4, 3)
            S[1, 2, 0] = 1 / torch.sqrt(torch.tensor(2.0))  # spin-raising operator (/sqrt(2))
            S[:, :, 2] = S[:, :, 0].T  # spin-lowering operator (/sqrt(2))
            # spin-z operator
            S[1, 1, 1] = +1 / 2
            S[2, 2, 1] = -1 / 2

            return [F, Z, S, I]

        else:  # spinless fermion
            # basis: empty, occupied
            F = torch.zeros(2, 2, 1)
            F[0, 1, 0] = 1

            Z = torch.diag(torch.tensor([1, -1]))

            return [F, Z, I]

    else:  # spin
        # basis: +s, +s-1, ..., -s
        Sp = torch.diag(torch.tensor([torch.sqrt((s - sp) * (s + sp + 1)) for sp in range(s - 1, -s - 1, -1)]), diagonal=1)  # spin raising operator
        Sz = torch.diag(torch.tensor([sp for sp in range(s, -s - 1, -1)]))  # spin-z operator
        S = torch.stack([Sp / torch.sqrt(torch.tensor(2.0)), Sz, Sp.T / torch.sqrt(torch.tensor(2.0))], dim=2)

        return [S, I]

# usage
# S, I = get_local_space('Spin', 1/2)

# Supplement : updateLeft(Zipper)

In [15]:
import torch

def update_left(Cleft, rankC, B, X, rankX, A):
    # sanity check
    if Cleft is None:
        rankC = 2  # regard as the case of the rank-2 identity
    if X is None:
        rankX = 2  # regard as the case of the rank-2 identity
    if rankC < len(Cleft.shape) or rankX < len(X.shape) or \
       [rankC, rankX] not in [[2, 2], [3, 2], [2, 3], [3, 3], [3, 4]]:
        raise ValueError('ERR: Invalid ranks of C and X.')

    B = torch.conj(B)  # take complex conjugate to B, without permuting legs

    if X is not None:
        T = torch.tensordot(X, A, dims=([2], [2]))

        if Cleft is not None:
            if rankC > 2 and rankX > 2:
                if rankX == 4:  # (rankC,rankX) = (3,4)
                    # contract the 3rd leg of Cleft and the 1st leg of X
                    T = torch.tensordot(Cleft, T, dims=([1, 2], [0, 1]))
                    Cleft = torch.tensordot(B, T, dims=([0, 2], [0, 1]))
                else:  # (rankC,rankX) = (3,3)
                    # contract the operator-flavor legs of Cleft and X
                    T = torch.tensordot(Cleft, T, dims=([1, 2], [0, 1]))
                    Cleft = torch.tensordot(B, T, dims=([0, 2], [0, 1]))
            else:  # (rankC,rankX) = (2,2), (2,3), (3,2)
                T = torch.tensordot(Cleft, T, dims=([1], [0]))
                Cleft = torch.tensordot(B, T, dims=([0, 2], [0, 1]))
        elif rankX == 4 and X.shape[0] == 1:  # no Cleft, rankX = 4
            # if X is rank-4 and its left leg is dummy, and Cleft is empty (i.e., identity)
            Cleft = torch.tensordot(B, T, dims=([0, 2], [0, 1]))
        else:  # no Cleft, rankX = 2,3
            Cleft = torch.tensordot(B, T, dims=([0, 2], [0, 1]))
    elif Cleft is not None:  # no X, rankC = 2, 3
        T = torch.tensordot(Cleft, A, dims=([1], [0]))
        Cleft = torch.tensordot(B, T, dims=([0, 2], [0, 1]))
    else:  # no Cleft, no X
        Cleft = torch.tensordot(B, A, dims=([0, 2], [0, 1]))
    return Cleft

# Tutorial 04.1 Diag Hamiltonian

# Excercise (b)

In [16]:
import torch

torch.manual_seed(0)  # for reproducibility

J = 1
N = 3

# Define spin operators and identity
S, I = getLocalSpace('Spin', 0.5)
Ss = [None] * N

# Define initial Hamiltonian and identity
H = torch.zeros(2, 2)
Aprev = torch.eye(2)

for itN in range(1, N+1):
    # Define identity tensor for the current iteration
    Anow = getIdentity(Aprev, 2, I, 2, [1, 3, 2])

    # Contract Hamiltonian up to the last iteration with ket tensor Anow and its Hermitian conjugate
    H = updateLeft(H, 2, Anow, None, None, Anow)

    for itN2 in range(1, itN):
        # Compute spin-spin interaction term
        Hsp = updateLeft(Ss[itN2-1], 3, Anow, S.conj().transpose(0, 1), 3, Anow)
        H += J * Hsp

    # Update spin operators to be represented in the new basis associated with the second leg of Anow
    for itN2 in range(1, itN+1):
        if itN2 < itN:
            Ss[itN2-1] = updateLeft(Ss[itN2-1], 3, Anow, None, None, Anow)
        else:
            Ss[itN2-1] = updateLeft(None, None, Anow, S, 3, Anow)

    # Replace Aprev with Anow for next iteration
    Aprev = Anow

# Diagonalize the Hamiltonian
Es = torch.sort(torch.eig((H+H.t()) / 2).eigenvalues.real, dim=0)[0]

print(Es)

RuntimeError: Tensors must have same number of dimensions: got 2 and 3